# Regulation Inspection

The goal is to get an understanding on how regulations work. 

My focus is on WX regulations.
According to the document "CRIDA-Regulations-20160401-20160430.csv", regulations have the following properties:

| dateReference | RegulationStart   | RegulationId | TrafficVolumeSet | ReferenceLocation | ReferenceLocationType | TrafficVolume | RegulationEnd     | RegulationActivity | RegulationCancelTime | RegulationDuration | AiracCycle | RegulationCategory | RegulationReasonCode | RegulationDescription | 
|---------------|-------------------|--------------|------------------|-------------------|-----------------------|---------------|-------------------|--------------------|----------------------|--------------------|------------|--------------------|----------------------|-----------------------| 
| 20160401      | 20160401 04:40:00 | EDDFA01      | EDGGFMP1         | EDDF              | AD                    | EDDFAWX       | 20160401 07:20:00 | C                  | 20160401 05:52:57    | 72                 | 411        | T                  | W                    | STRONG WINDS          | 
| 20160401      | 20160401 06:00:00 | LEBLA01M     | LECBFMP          | LEBL              | AD                    | LEBLARR       | 20160401 13:00:00 | C                  | 20160401 11:41:42    | 341                | 411        | T                  | W                    | CB + STORM            | 
| 20160401      | 20160401 08:40:00 | LECVN01M     | LECBFMP          | LECBCVN           | AS                    | LECBCVN1      | 20160401 11:20:00 | C                  | 20160401 08:16:03    | -23                | 411        | T                  | C                    |                       | 



Acc. "DatAcron Data sources for Flow Management scenarios.pptx", Traffic Volume is the sector name.

These are three typical examples for WX-caused regulations. It is important to note the column "referenceLocation" which indicates us that the critical weather, that caused the regulation, is not co-located with the regulated sector. Note also the "referenceLocationType", which denotes if the area of interest is an airport or an airspace.

The second important column is the "RegulationCancelTime", which denotes that a regulation may be cancelled if, for example, the actual weather turns out to be better than expected, or the network resilience is good enough to cope with demand exceeding capacity. The third regulation was even cancelled before it begun, yielding a negative "regulation time".



With the following data exploration, I hope to pull these important properties of the regulation out of the data store. 


In [1]:
import pandas as pd
from pandas.io.json import json_normalize, read_json
from SPARQLWrapper import SPARQLWrapper, JSON, XML, RDF

#import my connection module which allows to connect to the datAcron rdf store
from datacron_connector import TripleStoreConnector

ts107 = TripleStoreConnector(0)
ts109 = TripleStoreConnector(1)

Let's check what kind of regulations are stored in the triple store.

## Get all airblocks of spain

First, I will get all airblocks, together with their airspaces and super-airspaces.

In [5]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX myfn: <java:datAcronTester.unipi.gr.sparql_functions.>

SELECT  ?bigairspace ?airspace ?block ?wkt ?lowerlevel ?upperlevel
WHERE { 
   
   ?airspace dul:hasPart       ?block .
   ?block       :hasLowerLevel ?lowerlevel ;
                :hasUpperLevel ?upperlevel ;
                :hasGeometry   ?geom.
   ?geom        :hasMBR_WKT  ?wkt .
   
   OPTIONAL {?bigairspace dul:hasPart ?airspace. 
             #?conf           :configurationOfAirspace ?bigairspace ;
             #             dul:hasConstituent          ?t.
             #?t :TimeStart ?start ;
             #   :TimeEnd ?end.
             }.
   
   FILTER regex(str(?block), 'Airblock_LE', "i")
}
"""

df2 = ts109.query(qry)
df2 = ts109.clean(df2)
%time df2 = df2.sort_values('block')

df2.to_csv('data/allspanishairblocks.csv')
df2.head(10)

CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 25.9 ms


,bigairspace,airspace,block,wkt,lowerlevel,upperlevel
0,Airspace_LEABTMA_411,Airspace_LEABTA_411,Airblock_LEABTA_249LE,"POLYGON ((-2.99166666666667 39, -2.99166666666...",0.0,7467.6
1,Airspace_LETOT_411,Airspace_LEABTA_411,Airblock_LEABTA_249LE,"POLYGON ((-2.99166666666667 39, -2.99166666666...",0.0,7467.6
2,Airspace_LEAMTMA_411,Airspace_LEAMTA_411,Airblock_LEAMTA_650LE,"POLYGON ((-2.84083333333333 36.4166666666667, ...",0.0,4419.6
3,Airspace_LETOT_411,Airspace_LEAMTA_411,Airblock_LEAMTA_650LE,"POLYGON ((-2.84083333333333 36.4166666666667, ...",0.0,4419.6
4,Airspace_LEAMTMA_411,Airspace_LEAMTA_411,Airblock_LEAMTA_651LE,"POLYGON ((-1.83333333333333 36.85, -1.83333333...",0.0,4419.6
5,Airspace_LETOT_411,Airspace_LEAMTA_411,Airblock_LEAMTA_651LE,"POLYGON ((-1.83333333333333 36.85, -1.83333333...",0.0,4419.6
6,Airspace_LEAMTMA_411,Airspace_LEAMTA_411,Airblock_LEAMTA_652LE,"POLYGON ((-1.86111111111111 36.7138888888889, ...",0.0,4419.6
7,Airspace_LETOT_411,Airspace_LEAMTA_411,Airblock_LEAMTA_652LE,"POLYGON ((-1.86111111111111 36.7138888888889, ...",0.0,4419.6
9,Airspace_LETOT_411,Airspace_LEAMTA_411,Airblock_LEAMTA_653LE,"POLYGON ((-1.92361111111111 36.4530555555556, ...",0.0,4419.6
8,Airspace_LEAMTMA_411,Airspace_LEAMTA_411,Airblock_LEAMTA_653LE,"POLYGON ((-1.92361111111111 36.4530555555556, ...",0.0,4419.6


In [6]:
print(df2.iloc[0]['wkt'])

POLYGON ((-2.99166666666667 39, -2.99166666666667 39.4055555555556, -1.39444444444444 39.4055555555556, -1.39444444444444 39, -2.99166666666667 39))


In [17]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX myfn: <java:datAcronTester.unipi.gr.sparql_functions.>

SELECT  ?lowerlevel ?upperlevel ?s (myfn:getGeom(?s) as ?actualGeom)
WHERE { :Airblock_LEABTA_249LE :hasLowerLevel ?lowerlevel ;
                :hasUpperLevel ?upperlevel ;
                :hasGeometry   ?s.
}
"""

df3 = ts109.query(qry)
df3 = ts109.clean(df3)

df3.head()

,lowerlevel,upperlevel,s,actualGeom
0,0.0,7467.6,http://83.212.239.107/geometries/airblocks/249LE,"POLYGON ((-1.7525 39.4055555555556, -1.3944444..."


In [18]:
df3.iloc[0]['actualGeom']

'POLYGON ((-1.7525 39.4055555555556, -1.39444444444444 39, -1.48277777777778 39, -2.915 39, -2.99166666666667 39, -2.98333333333333 39.0666666666667, -2.9 39.2, -2.75 39.2666666666667, -1.7525 39.4055555555556))'

## Get all information about the three regulations above

First, I will check what kind of regulations are available and then, I will check what information is available on the three regulations from the example in the introduction.

In [2]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT ?s
WHERE {
  ?s rdfs:subClassOf* :FM_Regulation .
}
"""
 #?s rdf:type/rdfs:subClassOf* :SpatiotemporalRegion 

df = ts107.query(qry)
df = ts107.clean(df)
df.head(10)



URLError: <urlopen error [Errno 110] Connection timed out>

What properties are available for the WX regulations? I hope to see some of the columns listed in the introduction.

In [49]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT DISTINCT ?p
WHERE {
  ?s a :ATC_WeatherRegulation ; 
     ?p ?o .
}
"""
df = ts109.query(qry)
df = ts109.clean(df)

df

,p
0,type
1,hasRegion
2,hasTimeInterval
3,RegulationAiracCycle
4,RegulationDescription
5,hasReferenceLocation


PROBLEM

It seems that at least the 

 - reference location
 - reference location type
 - and the cancellation time 
 
are not available. Maybe hidden somewhere? I will inspect further. In the next query, I inspect all properties of the three regulations cited above.

In [4]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT *
WHERE {
  VALUES ?s {:EDDFA01_411 :LEBLA01M_411 :LECVN01M_411} 
  ?s dul:hasRegion             ?trafficVolume ;
        :hasReferenceLocation ?refLoc ;
 :RegulationDescription ?description ;
    dul:hasTimeInterval       ?time  ;
        a                     ?stype.

 ?time :TimeStart ?start ;
       :TimeEnd   ?end .
}
"""

reguls = ts109.query(qry)
reguls = ts109.clean(reguls)
reguls.to_csv('data/regulations.csv')
reguls.head(20)




,s,trafficVolume,refLoc,description,time,stype,start,end
0,EDDFA01_411,Airspace_EDDF_411,Place_Frankfurt_Main_Airport,STRONG WINDS_,intv_1459485600000_1459495200000,ATC_WeatherRegulation,2016-04-01T04:40:00,2016-04-01T05:52:57
1,LEBLA01M_411,Airspace_LEBL_411,Place_Barcelona_El_Prat_Airport,CB + STORM_,intv_1459490400000_1459515600000,ATC_WeatherRegulation,2016-04-01T06:00:00,2016-04-01T11:41:42
2,LECVN01M_411,Airspace_LECBCVN_411,Airspace_LECBCVN_411,_,intv_1459500000000_1459509600000,ATC_Capacity,2016-04-01T08:40:00,2016-04-01T08:16:03


In [8]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT *
WHERE {
  :Airspace_LETOT_411 ?p ?o
}
"""

space = ts109.query(qry)
space = ts109.clean(space)
space.head(20)



,p,o
0,type,FM_Airspace
1,hasPart,Airspace_LECBPP2_411
2,hasPart,Airspace_LECMDGU_411
3,hasPart,Airspace_LECMTLU_411
4,hasPart,Airspace_LECMPAU_411
5,hasPart,Airspace_GCCCAAC_411
6,hasPart,Airspace_LECMDGL_411
7,hasPart,Airspace_LECBP1U_411
8,hasPart,Airspace_LECBLVU_411
9,hasPart,Airspace_GCCCRNE_411


UPDATE

Good:
 - The referenceLocation are now encoded. 
 - The 109 yields the required results. The times are correct now.
 - But still entering the "sector" value from the regulations query does not get me any further.

To Do:
 - Somehow, I need to get from regulations to airblocks.
 - Maybe something wrong with my queries?
 - Here the answer from Giorgos: Comment 8: "Getting from regulations to airblocks is currently possible, with the following query:"

In [2]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX myfn: <java:datAcronTester.unipi.gr.sparql_functions.>

SELECT ?regulation ?airspace ?refloc (myfn:getGeom(?g) as ?WKT) ?low ?high ?config ?open ?close
WHERE {
  ?regulation      a             ?type .
  ?type       rdfs:subClassOf    :FM_Regulation .
  ?regulation  dul:hasRegion     ?airspace ;
                  :hasReferenceLocation ?refloc ;
               dul:hasTimeInterval       ?regtime  .
   ?regtime       :TimeStart             ?regstart ;
                  :TimeEnd               ?regend .

  ?airspace    dul:hasPart       ?s .
  ?s           dul:hasPart       ?b .
  ?b              :hasGeometry   ?g ;
                  :hasLowerLevel ?low ;
                  :hasUpperLevel ?high .
                  
                  
   ?config a :FM_Configuration ;
             :configurationOfAirspace ?airspace ;
          dul:hasConstituent          ?period .
   ?period   :TimeStart               ?open ;
             :TimeEnd                 ?close.
} LIMIT 1000

"""

regs_and_spaces = ts109.query(qry)
regs_and_spaces = ts109.clean(regs_and_spaces)
regs_and_spaces.head(60)

,regulation,airspace,refloc,WKT,low,high,config,open,close
0,LFMLE05E_411,Airspace_LFMLTMA_411,Airspace_LFMLTMA_411,"POLYGON ((5.18638888888889 43.4675, 5.21638888...",0.0,4419.6,AirspaceConfiguration_LFMLTMA_C1A_411,2016-03-31T00:00:00,2016-03-31T23:59:00
1,LFMLE05E_411,Airspace_LFMLTMA_411,Airspace_LFMLTMA_411,"POLYGON ((5.18638888888889 43.4675, 5.21638888...",0.0,4419.6,AirspaceConfiguration_LFMLTMA_C1A_411,2016-04-01T00:00:00,2016-04-01T23:59:00
2,LFMLE05E_411,Airspace_LFMLTMA_411,Airspace_LFMLTMA_411,"POLYGON ((5.18638888888889 43.4675, 5.21638888...",0.0,4419.6,AirspaceConfiguration_LFMLTMA_C1A_411,2016-04-02T00:00:00,2016-04-02T23:59:00
3,LFMLE05E_411,Airspace_LFMLTMA_411,Airspace_LFMLTMA_411,"POLYGON ((5.18638888888889 43.4675, 5.21638888...",0.0,4419.6,AirspaceConfiguration_LFMLTMA_C1A_411,2016-04-03T00:00:00,2016-04-03T23:59:00
4,LFMLE05E_411,Airspace_LFMLTMA_411,Airspace_LFMLTMA_411,"POLYGON ((5.18638888888889 43.4675, 5.21638888...",0.0,4419.6,AirspaceConfiguration_LFMLTMA_C1A_411,2016-04-04T00:00:00,2016-04-04T23:59:00
5,LFMLE05E_411,Airspace_LFMLTMA_411,Airspace_LFMLTMA_411,"POLYGON ((5.18638888888889 43.4675, 5.21638888...",0.0,4419.6,AirspaceConfiguration_LFMLTMA_C1A_411,2016-04-05T00:00:00,2016-04-05T23:59:00
6,LFMLE05E_411,Airspace_LFMLTMA_411,Airspace_LFMLTMA_411,"POLYGON ((5.18638888888889 43.4675, 5.21638888...",0.0,4419.6,AirspaceConfiguration_LFMLTMA_C1A_411,2016-04-06T00:00:00,2016-04-06T23:59:00
7,LFMLE05E_411,Airspace_LFMLTMA_411,Airspace_LFMLTMA_411,"POLYGON ((5.18638888888889 43.4675, 5.21638888...",0.0,4419.6,AirspaceConfiguration_LFMLTMA_C1A_411,2016-04-07T00:00:00,2016-04-07T23:59:00
8,LFMLE05E_411,Airspace_LFMLTMA_411,Airspace_LFMLTMA_411,"POLYGON ((5.18638888888889 43.4675, 5.21638888...",0.0,4419.6,AirspaceConfiguration_LFMLTMA_C1A_411,2016-04-08T00:00:00,2016-04-08T23:59:00
9,LFMLE05E_411,Airspace_LFMLTMA_411,Airspace_LFMLTMA_411,"POLYGON ((5.18638888888889 43.4675, 5.21638888...",0.0,4419.6,AirspaceConfiguration_LFMLTMA_C1A_411,2016-04-09T00:00:00,2016-04-09T23:59:00


In [3]:
len(regs_and_spaces)

1000

In [67]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX myfn: <java:datAcronTester.unipi.gr.sparql_functions.>

SELECT ?airspace ?refloc #(myfn:getGeom(?g) as ?WKT) ?low ?high
WHERE {
  :LEBLA01M_411 dul:hasRegion     ?airspace ;
                  :hasReferenceLocation ?refloc.
 # ?airspace    dul:hasPart       ?s .
#  ?s           dul:hasPart       ?b .
#  ?b              :hasGeometry   ?g ;
 #                 :hasLowerLevel ?low ;
#                  :hasUpperLevel ?high .
} limit 100

"""

regs_and_spaces = ts109.query(qry)
regs_and_spaces = ts109.clean(regs_and_spaces)
regs_and_spaces.head(60)

,airspace,refloc
0,Airspace_LEBL_411,Place_Barcelona_El_Prat_Airport


In [42]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX myfn: <java:datAcronTester.unipi.gr.sparql_functions.>

SELECT ?regulation ?airspace #(myfn:getGeom(?g) as ?WKT) 
WHERE {
  ?regulation      a             ?type .
  ?type       rdfs:subClassOf    :FM_Regulation .
  ?regulation  dul:hasRegion     ?airspace .
  ?airspace    dul:hasPart*      ?b .
  ?b              :hasGeometry   ?g
} limit 10

"""

regs_and_spaces = ts109.query(qry)
regs_and_spaces = ts109.clean(regs_and_spaces)
regs_and_spaces.head(20)


,regulation,airspace


Unfortunately, it does not seem to work out!


The next query used to misbehave before, but it is now fixed thanks to Giorgos Comment #9

In [68]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT ?s ?sector ?time ?refloc ?description ?start ?end ?duration
WHERE {
  VALUES ?s {:EDDFA01_411 :LEBLA01M_411 :LECVN01M_411 } 
  ?s dul:hasRegion             ?sector ;
        :hasReferenceLocation        ?refloc ;
        :RegulationDescription ?description ;
     dul:hasTimeInterval       ?time .
   
   ?time :TimeStart ?start ;
        :TimeEnd   ?end ; 
        :duration ?duration .

        
}
""" 

df = ts109.query(qry)
df = ts109.clean(df)
df.head(20)




,s,sector,time,refloc,description,start,end,duration
0,EDDFA01_411,Airspace_EDDF_411,intv_1459485600000_1459495200000,Place_Frankfurt_Main_Airport,STRONG WINDS_,2016-04-01T04:40:00,2016-04-01T05:52:57,72
1,LEBLA01M_411,Airspace_LEBL_411,intv_1459490400000_1459515600000,Place_Barcelona_El_Prat_Airport,CB + STORM_,2016-04-01T06:00:00,2016-04-01T11:41:42,341


In [45]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT ?p ?o
WHERE {
  :Place_Frankfurt_Main_Airport ?p ?o

        
}
""" 

df = ts109.query(qry)
df = ts109.clean(df)
df.head(20)




,p,o
0,type,major_Aerodrome
1,type,AerodromeClassA
2,hasGeometry,geom_8.571822869076076_50.05067708952074
3,partOfCountry,Place_Germany
4,elevationOfPlace,110.94720000000001
5,hasCountryCode,DE
6,hasPlaceName,"Flughafen Frankfurt Åm Main, Walldorf, Hesse, ..."
7,seeAlso,http://worldaerodata.com/wad.cgi?airport=EDDF
8,seeAlso,http://www.airportcity-frankfurt.com/
9,seeAlso,http://en.wikipedia.org/wiki/Frankfurt_Airport


# Weather Analysis (related to page 5 of tutorial) 

I think I will need some help here too!

Though we have seen that Sectors and Participants cannot be linked with other objects, I can infer from the semantics of the regulations that the airports EDDF and LEBL and the airspace LECB must have had severe wx at that time. 

Lets try to pull it out.

To my knowledge, the wx is written to the nodes, so I need to find the trajectories which have a spatio-temporal intersection with the wx event at the sector. For this, I begin with the basic intersection query from the tutorial, page 5.

In [22]:
# PREFIX bif: <java:datAcronTester.unipi.gr.sparql_functions.>   <--- only to be used in 109

qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?vessel ?time ?wkt ?speed  
WHERE {
?s1   a :Node ;
        :ofMovingObject ?vessel ;
        :hasSpeed ?speed ;
     dul:hasConstituent/:TimeStart ?time ;
     dul:hasConstituent/:hasWKT ?wkt .

FILTER(bif:st_distance(bif:st_geomfromtext ("POINT(13.139045 44.466133)"),bif:st_geomfromtext(?wkt))<=5)
}
"""

df = ts107.query(qry)
df = ts107.clean(df)
df.head(20)




EndPointInternalError: EndPointInternalError: endpoint returned code 500 and response. 

Response:
b'Virtuoso S1T00 Error SR171: Transaction timed out\n\nSPARQL query:\ndefine sql:big-data-const 0 \n#output-format:application/sparql-results+json\n\nPREFIX : <http://www.datacron-project.eu/datAcron#>\nPREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT ?vessel ?time ?wkt ?speed  \nWHERE {\n?s1   a :Node ;\n        :ofMovingObject ?vessel ;\n        :hasSpeed ?speed ;\n     dul:hasConstituent/:TimeStart ?time ;\n     dul:hasConstituent/:hasWKT ?wkt .\n\nFILTER(bif:st_distance(bif:st_geomfromtext ("POINT(13.139045 44.466133)"),bif:st_geomfromtext(?wkt))<=5)\n}\n'

PROBLEMS
 - I get a timeout on the 107 store
 - I get zero results on the 109 store
 - note: on the 107 store, the bif: prefix must not be used, on the 109 store, it must be used. Therefore, you cannot query both stores with the same string atm.
 
 
Back to EDDF and LEBL and LECB. Our goal is to get some weather for these places at regulation time. 

As a first step, we get the geometries of the _reference locations_.

In [23]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT DISTINCT ?airport ?iata ?placename ?geo ?o
WHERE {
  VALUES ?airport {'EDDF' 'LEBL' 'LECB'}
  ?s :hasICAOcode ?airport ;
     :hasIATAcode ?iata ;
     :hasPlaceName ?placename ;
     :hasGeometry ?geo .
  ?geo :hasMBR_WKT ?o .
     
}
"""

reflocs_manually = ts107.query(qry)
reflocs_manually = ts107.clean(reflocs_manually)
reflocs_manually.head(20)

,airport,iata,placename,geo,o
0,LEBL,BCN,BARCELONA/EL PRAT,geom_2_0783333778381348_41_29694366455078,POINT (2.0783333778381348 41.29694366455078)
1,EDDF,FRA,FRANKFURT MAIN,geom_8_570555686950684_50_03333282470703,POINT (8.570555686950684 50.03333282470703)


INFO

For the above query, once again, the 109 yields no results.


Next step: lets get time and position of Frankfurt airport and try to find some spatio-temporal intersecting trajectories.

In [24]:
time = reguls.iloc[0]['start']     #get the time of EDDF regulation
rpoint = reflocs_manually.iloc[0]['o'] #get the coordinates of Frankfurt airport (manually)

print(time)
print(rpoint)

2016-04-01T04:40:00
POINT (2.0783333778381348 41.29694366455078)


In the next query, I have to enter the times manually, because (as we have seen) the end time of a regulation is not available in the system. AFAIK, the nodes themselves only have start times, as they only represent a point in time.

In [28]:

qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?vessel ?time ?wkt ?speed 
WHERE {
    ?s1     a              :Node ;
           :ofMovingObject ?vessel ;
           :hasSpeed       ?speed ;
        dul:hasConstituent/:TimeStart ?time ;
        dul:hasConstituent/:hasWKT ?wkt .
        
    FILTER(bif:st_distance( bif:st_geomfromtext ('""" + rpoint + """'), bif:st_geomfromtext(?wkt)) <= 5 &&
    xsd:dateTime(substr(?time,1,19))>xsd:dateTime("2016-04-01 04:40:00") && 
    xsd:dateTime(substr(?time,1,19))<xsd:dateTime("2016-04-01 06:52:00"))
}
"""

df = ts107.query(qry)
df = ts107.clean(df)
df.head(20)





EndPointInternalError: EndPointInternalError: endpoint returned code 500 and response. 

Response:
b'Virtuoso S1T00 Error SR171: Transaction timed out\n\nSPARQL query:\ndefine sql:big-data-const 0 \n#output-format:application/sparql-results+json\n\nPREFIX : <http://www.datacron-project.eu/datAcron#>\nPREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n\nSELECT ?vessel ?time ?wkt ?speed \nWHERE {\n    ?s1     a              :Node ;\n           :ofMovingObject ?vessel ;\n           :hasSpeed       ?speed ;\n        dul:hasConstituent/:TimeStart ?time ;\n        dul:hasConstituent/:hasWKT ?wkt .\n        \n    FILTER(bif:st_distance( bif:st_geomfromtext (\'POINT (2.0783333778381348 41.29694366455078)\'), bif:st_geomfromtext(?wkt)) <= 5 &&\n    xsd:dateTime(substr(?time,1,19))>xsd:dateTime("2016-04-01 04:40:00") && \n    xsd:dateTime(substr(?time,1,19))<xsd:dateTime("2016-04-01 06:52:00"))\n}\n'

PROBLEM

 - According to this query, no trajectory intersects with EDDF.... I think this must be a query problem.
 - on the 107 store, this transaction times out.
 
 
 
Lets go on and check if wx conditions are available in the system.

In [35]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX sw: <http://sweet.jpl.nasa.gov>

SELECT ?s ?w
WHERE {
    ?s a             :Node ;
       :hasWeatherCondition ?w .
    
    FILTER regex(str(?s), 'm3', "i")
}
"""

df = ts109.query(qry)
df = ts109.clean(df)
df.head(5)




,s,w


LITTLE PROBLEM

Only in the 107 data store, I get results.

In [73]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX sw: <http://sweet.jpl.nasa.gov>

SELECT ?s ?p 
WHERE {
    ?s ?p :LEBLA01M_411
}
"""

df = ts109.query(qry)
df = ts109.clean(df)
df.head(5)




,s,p
0,traj_AA51136207_20160401082500_m2,affectedBy
1,traj_AA51134958_20160401101500_m2,affectedBy
2,traj_AA51143112_20160401090500_m2,affectedBy
3,traj_AA51142022_20160401064000_m2,affectedBy
4,traj_AA51144491_20160401080500_m2,affectedBy


In [2]:
df = ts109.qry_properties(':LEBLA01M_411', True)

df

,p,o
0,type,ATC_WeatherRegulation
1,hasRegion,Airspace_LEBL_411
2,hasTimeInterval,intv_1459490400000_1459515600000
3,RegulationAiracCycle,411
4,RegulationDescription,CB + STORM_
5,hasReferenceLocation,Place_Barcelona_El_Prat_Airport
